In [4]:
PATH = "/home/mdobrzanski/playground/IET/NLP/ustawy/"
%config Completer.use_jedi = False

In [71]:
import regex as re
import os
from morfeusz2 import Morfeusz 
import matplotlib.pyplot as plt
import elasticsearch
import elasticsearch_dsl
import os
import re
import spacy
import time
from collections import defaultdict, Counter, OrderedDict
from functools import reduce
import math
import pandas as pd
import requests
import collections

In [15]:
laws = {}

for filename in os.listdir(PATH):
    with open(PATH+filename, 'r') as file:
        tmp = file.read()
    laws[filename] = tmp

In [28]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [29]:
laws = {k: remove_html_tags(v) for k, v in laws.items()}

# 1

Uruchomione

In [30]:
res = requests.post('http://localhost:9200', data=laws['1996_146.txt'].encode('UTF-8'))

In [33]:
res = res.text.split('\n')

In [34]:
print(res)

['Dz\tnone', '\tdziennik\tbrev:pun\tdisamb', '.\tnone', '\t.\tinterp\tdisamb', 'U\tnone', '\tustawa\tbrev:pun\tdisamb', '.\tnone', '\t.\tinterp\tdisamb', 'z\tspace', '\tz\tprep:gen:nwok\tdisamb', '1996\tspace', '\t1996\tadj:sg:gen:m3:pos\tdisamb', 'r\tspace', '\trok\tbrev:pun\tdisamb', '.\tnone', '\t.\tinterp\tdisamb', '', 'Nr\tnewline', '\tnumer\tbrev:npun\tdisamb', '34\tspace', '\t34\tnum:pl:nom:m3:rec\tdisamb', ',\tnone', '\t,\tinterp\tdisamb', 'poz\tspace', '\tpozycja\tbrev:pun\tdisamb', '.\tnone', '\t.\tinterp\tdisamb', '146\tspace', '\t146\tadj:sg:nom:f:pos\tdisamb', '', 'USTAWA\tnone', '\tustawa\tsubst:sg:nom:f\tdisamb', '', 'z\tnone', '\tz\tprep:gen:nwok\tdisamb', 'dnia\tspace', '\tdzień\tsubst:sg:gen:m3\tdisamb', '2\tspace', '\t2\tadj:sg:gen:m3:pos\tdisamb', 'lutego\tspace', '\tluty\tsubst:sg:gen:m3\tdisamb', '1996\tspace', '\t1996\tadj:sg:gen:m3:pos\tdisamb', 'r\tspace', '\trok\tbrev:pun\tdisamb', '.\tnone', '\t.\tinterp\tdisamb', '', 'o\tnone', '\to\tprep:loc\tdisamb', 'zmia

# 2

# 3

In [43]:
laws_lemmatized = {}

for name, law in laws.items():
    res = requests.post('http://localhost:9200', data=law.encode('utf-8')).text
    res = [line for line in res.split('\n') if line != '']
    
    lemmatized = []
    for words in zip(res[0::2], res[1::2]):
        words = ' '.join(words)
        
        words = words.split()
        word = ' '.join(words[2:-2])
        category = words[-2]
        word = word.lower()
        category = category.split(':')[0]
        lemmatized += [f'{word}:{category}']
                
    laws_lemmatized[name] = lemmatized


In [46]:
laws_lemmatized['1996_146.txt']

['dziennik:brev',
 '.:interp',
 'ustawa:brev',
 '.:interp',
 'z:prep',
 '1996:adj',
 'rok:brev',
 '.:interp',
 'numer:brev',
 '34:num',
 ',:interp',
 'pozycja:brev',
 '.:interp',
 '146:adj',
 'ustawa:subst',
 'z:prep',
 'dzień:subst',
 '2:adj',
 'luty:subst',
 '1996:adj',
 'rok:brev',
 '.:interp',
 'o:prep',
 'zmiana:subst',
 'ustawa:subst',
 'o:prep',
 'utworzyć:ger',
 'agencja:subst',
 'rynek:subst',
 'rolny:adj',
 'oraz:conj',
 'niektóry:adj',
 'inny:adj',
 'ustawa:subst',
 'artykuł:brev',
 '.:interp',
 '1:adj',
 '.:interp',
 'w:prep',
 'ustawa:subst',
 'z:prep',
 'dzień:subst',
 '7:adj',
 'czerwiec:subst',
 '1990:adj',
 'rok:brev',
 '.:interp',
 'o:prep',
 'utworzyć:ger',
 'agencja:subst',
 'rynek:subst',
 'rolny:adj',
 '(:interp',
 'dziennik:brev',
 '.:interp',
 'ustawa:brev',
 '.:interp',
 'numer:brev',
 '39:num',
 ',:interp',
 'pozycja:brev',
 '.:interp',
 '223:adj',
 '):interp',
 'wprowadzać:fin',
 'się:qub',
 'następujący:adj',
 'zmiana:subst',
 '::interp',
 '1:adj',
 '):inter

# 4

In [47]:
def count_n_grams_text(text, n=2):
    result = defaultdict(int)
    for i in range(n,len(text)):
        result[' '.join(text[i-n:i])] += 1
    return result

def count_n_grams_dict(dictionary, n=2):
    result = list(map(lambda x: Counter(count_n_grams_text(x, n)), dictionary.values()))
    result = sum(result, Counter())
    return (defaultdict(int, result))

In [49]:
bigrams = count_n_grams_dict(laws_lemmatized, 2)

In [55]:
sorted(bigrams.items(), key=lambda b: b[1], reverse=True)

[('artykuł:brev .:interp', 83738),
 ('ustęp:brev .:interp', 53315),
 ('pozycja:brev .:interp', 45197),
 (',:interp pozycja:brev', 43168),
 ('.:interp 1:adj', 39986),
 ('-:interp -:interp', 36548),
 ('rok:brev .:interp', 32770),
 ('w:prep artykuł:brev', 32035),
 (',:interp o:prep', 29913),
 ('o:prep który:adj', 28656),
 ('który:adj mowa:subst', 28538),
 ('mowa:subst w:prep', 28473),
 ('.:interp 2:adj', 26408),
 ('w:prep ustęp:brev', 23536),
 ('.:interp artykuł:brev', 22917),
 (',:interp w:prep', 22544),
 ('.:interp numer:brev', 21433),
 ('1:adj .:interp', 21305),
 ('2:adj .:interp', 21118),
 (',:interp z:prep', 20015),
 ('):interp w:prep', 18691),
 ('.:interp .:interp', 18093),
 ('brzmienie:subst ::interp', 17128),
 ('.:interp 3:adj', 17071),
 ('_:interp _:interp', 16509),
 ('1:adj ):interp', 14071),
 ('.:interp w:prep', 14047),
 ('3:adj .:interp', 13736),
 (',:interp numer:brev', 13551),
 ('::interp 1:adj', 13534),
 ('2:adj ):interp', 12887),
 ('1:adj ,:interp', 12704),
 ('::interp ":i

# 5

In [74]:
def clean_n_grams(n_grams, n):
    to_del = []
    for k in n_grams.keys():
        if not k.replace(' ', '').replace('::', '????').replace(':', '').isalpha():
            to_del.append(k)
        elif len(k.split(' ')) != n:
            to_del.append(k)
    for k in to_del:
        del n_grams[k]

In [75]:
clean_n_grams(bigrams, 2)

In [76]:
sorted(bigrams.items(), key=lambda b: b[1], reverse=True)

[('w:prep artykuł:brev', 32035),
 ('o:prep który:adj', 28656),
 ('który:adj mowa:subst', 28538),
 ('mowa:subst w:prep', 28473),
 ('w:prep ustęp:brev', 23536),
 ('z:prep dzień:subst', 11360),
 ('otrzymywać:fin brzmienie:subst', 10535),
 ('określić:ppas w:prep', 9715),
 ('do:prep sprawa:subst', 8718),
 ('ustawa:subst z:prep', 8625),
 ('właściwy:adj do:prep', 8535),
 ('i:conj numer:brev', 8435),
 ('dodawać:fin się:qub', 8196),
 ('minister:subst właściwy:adj', 7933),
 ('w:prep brzmienie:subst', 7277),
 ('w:prep droga:subst', 7128),
 ('w:prep przypadek:subst', 6776),
 ('na:prep podstawa:subst', 6681),
 ('stosować:fin się:qub', 6535),
 ('się:qub wyraz:subst', 6077),
 ('w:prep zakres:subst', 5476),
 ('od:prep dzień:subst', 5324),
 ('w:prep ustawa:subst', 5276),
 ('zastępować:fin się:qub', 4779),
 ('droga:subst rozporządzenie:subst', 4748),
 ('móc:fin być:inf', 4515),
 ('w:prep sprawa:subst', 4500),
 ('oraz:conj z:prep', 4302),
 ('w:prep życie:subst', 4289),
 ('w:prep termin:subst', 4190),
 ('

# 6

# 7

In [77]:
def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    counts = list(counts)
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k/total + (k==0)) for k in counts])


def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11+k12, k21+k22]) +
                denormEntropy([k11+k21, k12+k22]) -
                denormEntropy([k11, k12, k21, k22]))

In [78]:
def count_token_pos(n_grams, n):
    n_grams_token_pos = {i: defaultdict(int) for i in range(n)}
    for ngram, count in n_grams.items():
        tokens = ngram.split(' ')
        for i, t in enumerate(tokens):
            n_grams_token_pos[i][t] += count
    return n_grams_token_pos

In [79]:
bigram_count = sum(bigrams.values()) 

In [80]:
first_count, second_count = count_token_pos(bigrams, 2).values()

In [81]:
bigrams_llr = {}

for bigram, count in bigrams.items():
    first_token, second_token = bigram.split(' ')
    k11 = count
    k12 = second_count[second_token] - count
    k21 = first_count[first_token] - count
    k22 = bigram_count - (k11 + k12 + k21)
    bigrams_llr[bigram] = llr_2x2(k11, k12, k21, k22)

In [82]:
bigrams_llr = OrderedDict(sorted(bigrams_llr.items(), key=lambda x: x[1], reverse=True))

In [83]:
list(bigrams_llr.items())

[('który:adj mowa:subst', 247306.84252548264),
 ('o:prep który:adj', 189881.37388472375),
 ('mowa:subst w:prep', 176270.37764490908),
 ('w:prep artykuł:brev', 113020.91939746356),
 ('otrzymywać:fin brzmienie:subst', 110379.19221575293),
 ('w:prep ustęp:brev', 87666.64992744755),
 ('minister:subst właściwy:adj', 70603.56428862398),
 ('dodawać:fin się:qub', 66315.11262953095),
 ('i:conj numer:brev', 54712.87883012241),
 ('droga:subst rozporządzenie:subst', 53735.28148106733),
 ('z:prep dzień:subst', 53496.1935577048),
 ('stosować:fin się:qub', 52561.12023337511),
 ('określić:ppas w:prep', 51801.50598787842),
 ('właściwy:adj do:prep', 50095.68621733773),
 ('na:prep podstawa:subst', 47439.4810785068),
 ('do:prep sprawa:subst', 46340.12044411001),
 ('móc:fin być:inf', 41765.48532452018),
 ('ustawa:subst z:prep', 41615.609246767126),
 ('rzeczpospolita:subst polski:adj', 41419.802977751824),
 ('się:qub wyraz:subst', 40299.276388325496),
 ('a:conj także:conj', 40259.013534228085),
 ('zastępowa

# 8

In [85]:
partitions = collections.defaultdict(dict)

for k, v in bigrams.items():
    w1, w2 = k.split(' ')
    w1, cat1 = w1.split(':')
    w2, cat2 = w2.split(':')
    partitions[(cat1, cat2)][k] = v

In [88]:
list(partitions.keys())

[('subst', 'prep'),
 ('prep', 'subst'),
 ('subst', 'subst'),
 ('prep', 'ger'),
 ('ger', 'subst'),
 ('subst', 'adj'),
 ('adj', 'conj'),
 ('conj', 'adj'),
 ('adj', 'adj'),
 ('adj', 'subst'),
 ('subst', 'brev'),
 ('fin', 'qub'),
 ('qub', 'adj'),
 ('prep', 'brev'),
 ('qub', 'brev'),
 ('subst', 'fin'),
 ('fin', 'inf'),
 ('inf', 'subst'),
 ('fin', 'subst'),
 ('fin', 'adj'),
 ('conj', 'subst'),
 ('subst', 'ppas'),
 ('ppas', 'prep'),
 ('prep', 'adj'),
 ('fin', 'prep'),
 ('subst', 'conj'),
 ('ger', 'conj'),
 ('conj', 'ger'),
 ('ger', 'adj'),
 ('adj', 'prep'),
 ('conj', 'ppas'),
 ('adj', 'pact'),
 ('pact', 'subst'),
 ('subst', 'adja'),
 ('subst', 'ger'),
 ('subst', 'pact'),
 ('pact', 'prep'),
 ('ger', 'ppas'),
 ('adj', 'fin'),
 ('subst', 'bedzie'),
 ('bedzie', 'inf'),
 ('subst', 'pcon'),
 ('pcon', 'subst'),
 ('fin', 'ppas'),
 ('ppas', 'conj'),
 ('conj', 'conj'),
 ('qub', 'subst'),
 ('conj', 'fin'),
 ('fin', 'conj'),
 ('pact', 'ger'),
 ('adv', 'prep'),
 ('fin', 'adv'),
 ('pact', 'qub'),
 ('qub', 

# 9

In [93]:
partition_count = {}

for p, bigrams in partitions.items():
    s = 0
    for _, count in bigrams.items():
        s += count
    
    partition_count[p] = s

In [109]:
top_10_cat = sorted(partition_count.items(), key=lambda p: p[1], reverse=True)[:10]

In [110]:
top_10_cat

[(('prep', 'subst'), 326523),
 (('subst', 'subst'), 292416),
 (('subst', 'adj'), 273411),
 (('adj', 'subst'), 187562),
 (('subst', 'prep'), 173132),
 (('subst', 'conj'), 84881),
 (('conj', 'subst'), 83923),
 (('prep', 'adj'), 79171),
 (('ger', 'subst'), 76137),
 (('prep', 'brev'), 66877)]

# 10

In [122]:
for p in top_10_cat:
    print(p[0])
    subset = partitions[p[0]]
    top_5_llr = sorted(subset.items(), key=lambda p: bigrams_llr[p[0]], reverse=True)[:5]
    print('\n'.join([str(k) for k in top_5_llr]), '\n\n')

('prep', 'subst')
('z:prep dzień:subst', 11360)
('na:prep podstawa:subst', 6681)
('do:prep sprawa:subst', 8718)
('w:prep droga:subst', 7128)
('od:prep dzień:subst', 5324) 


('subst', 'subst')
('droga:subst rozporządzenie:subst', 4748)
('skarb:subst państwo:subst', 1821)
('rada:subst minister:subst', 2268)
('terytorium:subst rzeczpospolita:subst', 1224)
('ochrona:subst środowisko:subst', 1572) 


('subst', 'adj')
('minister:subst właściwy:adj', 7933)
('rzeczpospolita:subst polski:adj', 3578)
('jednostka:subst organizacyjny:adj', 2252)
('samorząd:subst terytorialny:adj', 1675)
('produkt:subst leczniczy:adj', 1738) 


('adj', 'subst')
('który:adj mowa:subst', 28538)
('niniejszy:adj ustawa:subst', 2364)
('następujący:adj zmiana:subst', 1624)
('odrębny:adj przepis:subst', 1449)
('walny:adj zgromadzenie:subst', 598) 


('subst', 'prep')
('mowa:subst w:prep', 28473)
('ustawa:subst z:prep', 8625)
('wniosek:subst o:prep', 2619)
('dzień:subst od:prep', 2510)
('miesiąc:subst od:prep', 1581) 


(

# 11

i. Prawie wszystkie (8/10) największe grupy zawierały rzeczownik

ii. Składające się jedynie z rzeczowników, przymiotników i czasowników

iii. Myślę że najlepsze jest połączenio obydwu, jeśli mamy tylko jedno to chyba llr

iv. Tłumaczenia, analiza sentymentów